<h1>Data Preprocessing for Plotly Exploration</h1>

<h1>Data Selection</h1>

<p>Data sources and data description <br>
New York City has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood. This dataset exists here: https://geo.nyu.edu/catalog/nyu_2451_34572</p>

<p>Foursquare has been compiling and analyzing COVID-19 Foot Traffic Data since 02/19/2020 and it can be interfaced with through amazon web services. This data set contains indexed foot traffic to 19 categories of venues. The indexed data is broken out geographically, with included data for National, SF, NYC, LA, and Seattle. The data is normalized against U.S. Census data to remove age, gender and geographical bias. They use indexed foot traffic to demonstrate the relative decline in visits to different types of places, where visits on the first day are 100. They analyze data on a rolling 7-day basis to reduce the effects of foot traffic trends influenced by certain days of the week (for example, bars and clubs experience an uptick on Fridays and Saturdays).</p>

<p>So, to explain this in an example, an 81 index to airports in Seattle for March 6 indicates that foot traffic between February 29 - March 6 (a 7-day rolling period) is 19% lower than the first 7 days of analysis, February 13 - February 19.</p>

<p>They used February 13 - February 19 as the first 7-day period benchmark for analysis because February 19 is when they last estimated foot traffic to be roughly normal for the categories analyzed.</p> 

<p>We will also be interfacing with the foursquare api to pull venue information for New York City. For reproduction purposes we will stay within the free developer account parameters:<br> 
<ul><li>99,500 Regular calls / day</li>
<li>500 Premium calls / day</li>
<li>2 Photos per venue</li>
<li>2 Tips per venue</li></ul> 

<p>These exploration calls will yield:
 <ul><li>Venue name</li>
     <li>A unique benue Id</li>
<li>Venue category(s)</li>
<li>Venue location attributes such as neighborhood, latitude, longitude, and postal address</li>
<li>Venue popularity</li></ul>

<p>Process Steps:</p>
<ul><li>Import data from Foursquare Covid-19 Reports</li>
    <li>Wrangle data from Foursquare Covid-19 Reports</li>
    <li>Import NYC Borough data</li>
    <li>Map NYC</li>
    <li>Import NYC venue data from Foursquare API</li>
    <li>Filter and organize the data for the categories of focus</li>
    <li>Create a new dataframe that extrapolates on the daily foot traffic impact on the venues.</li>
    </ul>

<h3>Process Step: import libraries needed</h3>

In [1]:
!pip install folium

In [2]:
# importing libraries needed
import numpy as np # library to handle data in a vectorized manner
import requests # library to handle requests

import copy # for deep copying
from datetime import datetime # for file saving extensions 

import types
import pandas as pd # library for data analsysis
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import json # library to handle JSON files
import collections # library for ordered dictionary
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium # map rendering library
import openpyxl


print('Libraries imported.')

Libraries imported.


<h3>Process Step: import and parse Foursquare COVID-19 Data National + DMA_March_2_2021</h3>

In [3]:
# The code was removed by Watson Studio for sharing.

,Date,Airports,Auto Shops,Banks,Bars,Big Box Stores,Discount Stores,Casual Dining Chains,Clothing Stores,Convenience Store,...,Hardware Stores,Hotels,Liquor Stores,Movie Theaters,Nail Salons,Offices,Parks,SalonsBarbershops,Shopping Malls,Warehouse Stores
0,2020-02-19,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.000000,100.000000,100.000000,100.000000,NaN,100.000000,100.000000,100.000000,100.000000,100.000000
1,2020-02-20,99.094944,102.542650,100.625851,100.420756,97.394488,95.556406,101.048893,99.607061,98.274066,...,101.362987,100.737405,99.986668,99.918510,NaN,99.949037,100.722542,99.845995,99.194796,100.539919
2,2020-02-21,97.147890,102.404818,100.114900,99.909588,96.693740,93.407566,97.998888,98.069669,96.400531,...,102.640193,100.395186,96.785253,96.391643,NaN,100.254692,100.526705,98.846516,97.497906,100.629173
3,2020-02-22,96.677029,104.128687,99.761536,102.007137,96.756355,94.702510,98.426706,98.312451,96.866640,...,103.668323,99.149662,99.628367,94.546198,NaN,100.529350,106.851461,100.480999,97.182439,101.988823
4,2020-02-23,97.675829,104.913415,99.483330,100.557640,96.877462,93.895229,95.934193,97.041579,97.173642,...,104.061688,94.992057,98.292465,90.478557,NaN,100.075248,114.429285,101.035450,95.539241,103.387569


In [6]:
#importing Foursquare COVID-19 Data National + DMA_March_2_2021.xlsx to df_4s_nat_dma
nyc = df_data_1
nyc.head()

,Date,Airports,Auto Shops,Banks,Bars,Big Box Stores,Discount Stores,Casual Dining Chains,Clothing Stores,Convenience Store,...,Hardware Stores,Hotels,Liquor Stores,Movie Theaters,Nail Salons,Offices,Parks,SalonsBarbershops,Shopping Malls,Warehouse Stores
0,2020-02-19,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.000000,100.000000,100.000000,100.000000,NaN,100.000000,100.000000,100.000000,100.000000,100.000000
1,2020-02-20,99.094944,102.542650,100.625851,100.420756,97.394488,95.556406,101.048893,99.607061,98.274066,...,101.362987,100.737405,99.986668,99.918510,NaN,99.949037,100.722542,99.845995,99.194796,100.539919
2,2020-02-21,97.147890,102.404818,100.114900,99.909588,96.693740,93.407566,97.998888,98.069669,96.400531,...,102.640193,100.395186,96.785253,96.391643,NaN,100.254692,100.526705,98.846516,97.497906,100.629173
3,2020-02-22,96.677029,104.128687,99.761536,102.007137,96.756355,94.702510,98.426706,98.312451,96.866640,...,103.668323,99.149662,99.628367,94.546198,NaN,100.529350,106.851461,100.480999,97.182439,101.988823
4,2020-02-23,97.675829,104.913415,99.483330,100.557640,96.877462,93.895229,95.934193,97.041579,97.173642,...,104.061688,94.992057,98.292465,90.478557,NaN,100.075248,114.429285,101.035450,95.539241,103.387569


In [7]:
# let's replace the index with dates and then set it as the index
nyc.set_index('Date',drop=True,inplace=True)
nyc.head()

,Airports,Auto Shops,Banks,Bars,Big Box Stores,Discount Stores,Casual Dining Chains,Clothing Stores,Convenience Store,Drugstores & Pharmacies,...,Hardware Stores,Hotels,Liquor Stores,Movie Theaters,Nail Salons,Offices,Parks,SalonsBarbershops,Shopping Malls,Warehouse Stores
Date,,,,,,,,,,,,,,,,,,,,,
2020-02-19,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.000000,100.000000,100.000000,100.000000,NaN,100.000000,100.000000,100.000000,100.000000,100.000000
2020-02-20,99.094944,102.542650,100.625851,100.420756,97.394488,95.556406,101.048893,99.607061,98.274066,95.419486,...,101.362987,100.737405,99.986668,99.918510,NaN,99.949037,100.722542,99.845995,99.194796,100.539919
2020-02-21,97.147890,102.404818,100.114900,99.909588,96.693740,93.407566,97.998888,98.069669,96.400531,90.788783,...,102.640193,100.395186,96.785253,96.391643,NaN,100.254692,100.526705,98.846516,97.497906,100.629173
2020-02-22,96.677029,104.128687,99.761536,102.007137,96.756355,94.702510,98.426706,98.312451,96.866640,90.648260,...,103.668323,99.149662,99.628367,94.546198,NaN,100.529350,106.851461,100.480999,97.182439,101.988823
2020-02-23,97.675829,104.913415,99.483330,100.557640,96.877462,93.895229,95.934193,97.041579,97.173642,90.838629,...,104.061688,94.992057,98.292465,90.478557,NaN,100.075248,114.429285,101.035450,95.539241,103.387569


In [8]:
#let's check for null values
for col in nyc:
    print(col,' has ',nyc[col].isnull().sum(),'NaN values.')

Airports  has  0 NaN values.
Auto Shops  has  0 NaN values.
Banks  has  0 NaN values.
Bars  has  0 NaN values.
Big Box Stores  has  0 NaN values.
Discount Stores  has  0 NaN values.
Casual Dining Chains  has  307 NaN values.
Clothing Stores  has  0 NaN values.
Convenience Store  has  0 NaN values.
Drugstores & Pharmacies  has  0 NaN values.
Fast Food  has  0 NaN values.
Furniture Stores  has  0 NaN values.
Gas Stations  has  0 NaN values.
Grocery Stores  has  0 NaN values.
Gyms  has  0 NaN values.
Hardware Stores  has  0 NaN values.
Hotels  has  0 NaN values.
Liquor Stores  has  0 NaN values.
Movie Theaters  has  328 NaN values.
Nail Salons  has  366 NaN values.
Offices  has  0 NaN values.
Parks  has  0 NaN values.
SalonsBarbershops  has  0 NaN values.
Shopping Malls  has  0 NaN values.
Warehouse Stores  has  0 NaN values.


<h3>Process Step: Make choices about the Foursquare's COVID-19 Impact data</h3>
<p>After reviewing the null values from Foursquare's COVID-19 Impact data we will make the following choices:<ul><li> Drop Movie Theaters because they have been closed throughout COVID-19.</li><li>Drop Nail Saloons for two reasons:(1)we only have national data and New York City has atypical foot traffic pattern for Nail Saloons (2) Nail Saloons have only recently opened with strict safety protocols.</li><li>As of March 18, Casual Dining Chains stopped reporting their data. So we will take the average index score for Casual Dining Chains for the proceeding 4 weeks and assign it to the NaN values.</ul></p>

In [9]:
nyc= nyc.drop(labels=['Movie Theaters','Nail Salons'],axis=1 )
nyc.columns

Index(['Airports', 'Auto Shops', 'Banks', 'Bars', 'Big Box Stores',
       'Discount Stores', 'Casual Dining Chains', 'Clothing Stores',
       'Convenience Store', 'Drugstores & Pharmacies', 'Fast Food',
       'Furniture Stores', 'Gas Stations', 'Grocery Stores', 'Gyms',
       'Hardware Stores', 'Hotels', 'Liquor Stores', 'Offices', 'Parks',
       'SalonsBarbershops', 'Shopping Malls', 'Warehouse Stores'],
      dtype='object')

In [10]:
#get loc for start of mean calc (27)
s = nyc.index.get_loc('2020-03-17')
#get loc for end of mean calc (58)
e = nyc.index.get_loc('2020-04-17')

nyc_dining_slice = nyc['Casual Dining Chains'].iloc[s:e]
nyc_dining_slice.mean() #32.284104665696574

#let's fill the NaN values with 32.284104665696574
nyc['Casual Dining Chains'] = nyc['Casual Dining Chains'].fillna(32.284104665696574)

#let's check for null values again
for col in nyc:
    print(col,' has ',nyc[col].isnull().sum(),'NaN values.')

Airports  has  0 NaN values.
Auto Shops  has  0 NaN values.
Banks  has  0 NaN values.
Bars  has  0 NaN values.
Big Box Stores  has  0 NaN values.
Discount Stores  has  0 NaN values.
Casual Dining Chains  has  0 NaN values.
Clothing Stores  has  0 NaN values.
Convenience Store  has  0 NaN values.
Drugstores & Pharmacies  has  0 NaN values.
Fast Food  has  0 NaN values.
Furniture Stores  has  0 NaN values.
Gas Stations  has  0 NaN values.
Grocery Stores  has  0 NaN values.
Gyms  has  0 NaN values.
Hardware Stores  has  0 NaN values.
Hotels  has  0 NaN values.
Liquor Stores  has  0 NaN values.
Offices  has  0 NaN values.
Parks  has  0 NaN values.
SalonsBarbershops  has  0 NaN values.
Shopping Malls  has  0 NaN values.
Warehouse Stores  has  0 NaN values.


<h3>Process Step: Get the NYC borough data</h3>

In [11]:
!wget -q -O 'nyu_2451_34572-geojson.json' https://cocl.us/new_york_dataset

In [12]:
# get the NYC borough data you need the wget software
!wget -q -O 'nyu_2451_34572-geojson.json' https://cocl.us/new_york_dataset
print('Data downloaded!')
with open('nyu_2451_34572-geojson.json') as json_data:
    newyork_data = json.load(json_data)
    
neighborhoods_data = newyork_data['features']
# Transform the data into a *pandas* dataframe
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe 
neighborhoods = pd.DataFrame(columns=column_names)
# Then let's loop through the data and fill the dataframe one row at a time.
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
print('neighborhoods dataframe created!')

Data downloaded!
neighborhoods dataframe created!


In [13]:
# Let's inspect the pandas data frame
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


<p>For illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in Manhattan. So let's slice the original dataframe and create a new dataframe of the Manhattan data.</p>

In [14]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


#### Use geopy library to get the latitude and longitude values of New York City.

In [15]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

# create map of NYC using latitude and longitude values
map_nyc = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Borough'], manhattan_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        ).add_to(map_nyc)  
    
map_nyc

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


<h3>Process Step: Pull Down Exploration Data From Fourquare API</h3>

In [16]:
# The code was removed by Watson Studio for sharing.

<h4>Create category ID variables to load into lists and dictionaries for filtering</h4>

In [17]:
#Fousquare Venue Category ID
Airports="4bf58dd8d48988d1ed931735"
AutoShops="56aa371be4b08b9a8d5734d3"
Banks="4bf58dd8d48988d10a951735"
Bars="4bf58dd8d48988d116941735"
BigBoxStores="52f2ab2ebcbc57f1066b8b42"
CasualDiningChains=""
ClothingStores="4bf58dd8d48988d103951735"
ConvenienceStore="4d954b0ea243a5684a65b473"
DrugstoresPharmacies="5745c2e4498e11e7bccabdbd"
DiscountStores="52dea92d3cf9994f4e043dbb"
FastFood="4bf58dd8d48988d16e941735"
GasStations="4bf58dd8d48988d113951735"
GroceryStores="4bf58dd8d48988d118951735"
Gyms="4bf58dd8d48988d175941735"
HardwareStores="4bf58dd8d48988d112951735"
Hotels="4bf58dd8d48988d1fa931735"
LiquorStores="4bf58dd8d48988d186941735"
Offices="4bf58dd8d48988d124941735"
Parks="4bf58dd8d48988d163941735"
SalonsBarbershops="4bf58dd8d48988d110951735"
ShoppingMalls="4bf58dd8d48988d1fd941735"
WarehouseStores="52e816a6bcbc57f1066b7a54"  

#Venue Categories List
Category_list=[Airports,AutoShops,Banks,Bars,BigBoxStores,ClothingStores,ConvenienceStore,DrugstoresPharmacies,DiscountStores,FastFood,GasStations,GroceryStores,Gyms,HardwareStores,Hotels,LiquorStores,Offices,Parks,SalonsBarbershops,ShoppingMalls,WarehouseStores]
strcat = ','.join(str(e) for e in Category_list)

In [18]:
#Creating a dictionary to convert foursquare categories back to the index names
cat_list_convert_to_dict = list(nyc.columns)
cat_list_convert_to_dict.remove('Casual Dining Chains')
cat_dict=dict([[y,cat_list_convert_to_dict[x]] for x,y in enumerate(Category_list)])
cat_dict['4bf58dd8d48988d110951735']='Salons & Barbershops'
cat_dict

{'4bf58dd8d48988d1ed931735': 'Airports',
 '56aa371be4b08b9a8d5734d3': 'Auto Shops',
 '4bf58dd8d48988d10a951735': 'Banks',
 '4bf58dd8d48988d116941735': 'Bars',
 '52f2ab2ebcbc57f1066b8b42': 'Big Box Stores',
 '4bf58dd8d48988d103951735': 'Discount Stores',
 '4d954b0ea243a5684a65b473': 'Clothing Stores',
 '5745c2e4498e11e7bccabdbd': 'Convenience Store',
 '52dea92d3cf9994f4e043dbb': 'Drugstores & Pharmacies',
 '4bf58dd8d48988d16e941735': 'Fast Food',
 '4bf58dd8d48988d113951735': 'Furniture Stores',
 '4bf58dd8d48988d118951735': 'Gas Stations',
 '4bf58dd8d48988d175941735': 'Grocery Stores',
 '4bf58dd8d48988d112951735': 'Gyms',
 '4bf58dd8d48988d1fa931735': 'Hardware Stores',
 '4bf58dd8d48988d186941735': 'Hotels',
 '4bf58dd8d48988d124941735': 'Liquor Stores',
 '4bf58dd8d48988d163941735': 'Offices',
 '4bf58dd8d48988d110951735': 'Salons & Barbershops',
 '4bf58dd8d48988d1fd941735': 'SalonsBarbershops',
 '52e816a6bcbc57f1066b7a54': 'Shopping Malls'}

<h4>Create a function to explore all the neighborhoods in Manhattan</h4>

In [19]:
# The code was removed by Watson Studio for sharing.

In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
        strcat)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['id'],
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category','Venue Category Name']

            
    return(nearby_venues)

In [21]:
#run the function and create dataframe manhattan_venues
nyc_v = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [25]:
for v in nyc_v['Venue Category']:
    if v in cat_dict.keys():
        nyc_v.replace(to_replace = v, value =cat_dict[v],inplace=True)

In [26]:
print('There are {} uniques categories.'.format(len(nyc_v['Venue Category'].unique())))

There are 146 uniques categories.


<p>Fourquare considers the child category the primary cateogy. We will have to filter our data until for the category IDs we can attribute to the COVID-19 Impact Data. We can manually edit the child categories later if needed.</p>

In [32]:
filt =list(cat_dict.values()) # create list to filter through
v_cat_named = nyc_v[nyc_v['Venue Category'].isin(filt)] #filter the dataset

# Need to conventionalize Salon & barbershop naming
v_cat_named = v_cat_named.replace("SalonsBarbershops","Salons & Barbershops")
nyc.rename(columns={"SalonsBarbershops":"Salons & Barbershops"},inplace=True)
print('There are {} uniques categories.'.format(len(v_cat_named['Venue Category'].unique())))



There are 20 uniques categories.


<h3>Process Step: Extrapolate the venue data so that each venue has an index score each day.</h3>
<p> To do this we will create an ordered dictionary that will be filled by a nested loop that will itterate each row of the Foursquare Venue dataframe and the COVID-19 Impact dataframe. We will use a deep copy so that we don't effect the original dataframes and we will save the exported dataframe to an Excel document.</p>

In [78]:
#nyc.reset_index(inplace=True).rename(columns={"index": "Date"}) # alternate if name attribute doesn't work

2020-02-19 00:00:00


In [34]:
# Do not run this multiple times
# creating an ordered dictionary to loop through the dataframes and extrapolate
keys=['Date','Score','Neighborhood', 'Neighborhood Latitude','Neighborhood Longitude', 'Venue', 'Venue Category','Venue Latitude', 'Venue Longitude']
od_venues=collections.OrderedDict({key:[] for key in keys})

for place in v_cat_named.index:
    cat=str(v_cat_named.loc[place,'Venue Category'])
    for (i, row) in nyc.iterrows():
        od_venues['Date'].append(row.name)
        od_venues['Score'].append(row[cat])
        od_venues['Neighborhood'].append(v_cat_named.loc[place,'Neighborhood'])
        od_venues['Neighborhood Latitude'].append(v_cat_named.loc[place,'Neighborhood Latitude'])
        od_venues['Neighborhood Longitude'].append(v_cat_named.loc[place,'Neighborhood Longitude'])
        od_venues['Venue'].append(v_cat_named.loc[place,'Venue'])
        od_venues['Venue Latitude'].append(v_cat_named.loc[place,'Venue Latitude'])
        od_venues['Venue Longitude'].append(v_cat_named.loc[place,'Venue Longitude'])
        od_venues['Venue Category'].append(v_cat_named.loc[place,'Venue Category'])

#Save the dataframe for backup
# datetime object containing current date and time
now = datetime.now()
dt_string = now.strftime("%d.%m.%Y_%H.%M.%S")
df_r4_plot=pd.DataFrame(od_venues, columns=od_venues.keys())
#df_r4_plot.to_csv('fs_covid19_df_ready_for_plot'+dt_string+'.csv')
print('Dataframe made and saved!')

Dataframe made and saved!
